In [70]:
import numpy as np

In [83]:
class NeuronLayer:
    def __init__(self, count, prevCount, first=False):
        self.count = count
        self.prevCount = prevCount
        self.randNormChoices = 
        np.random.shuffle(self.randNormChoices)
        self.neurons = None
        self.errors = None
        if(not first):
            self.synapses = np.random.choice(
                                np.append(np.random.normal(-0.5, 0.1, 
                                    np.max([self.count, self.prevCount])),
                                np.random.normal(0.5, 0.1, 
                                    np.max([self.count, self.prevCount]))), 
                                (self.count, self.prevCount)) 
        del self.randNormChoices

In [106]:
class NeuralNetwork:
    def __init__(self):
        self.layers = 0
        self.network = list()

    def initNetwork(self, networkStruct):
        self.layers = len(networkStruct)
        self.network.append(NeuronLayer(networkStruct[0], 0, True))
        for i in range(self.layers-1):
            self.network.append(NeuronLayer(networkStruct[i+1], networkStruct[i]))

    def feedForward(self, data):
        self.network[0].neurons = data.reshape(self.network[0].count, 1)
        for i in range(self.layers-2):
            self.network[i+1].neurons = self.tanh(np.dot(self.network[i+1].synapses, 
                                                         self.network[i].neurons))
        self.network[-1].neurons = self.softmax(np.dot(self.network[-1].synapses, 
                                                       self.network[-2].neurons))

    def backpropagation(self, target, learningRate):
        self.network[-1].errors = target - self.network[-1].neurons
        for i in reversed(range(1, self.layers)):
            self.network[i].synapses += learningRate * np.dot(self.network[i].errors, 
                                                              self.network[i-1].neurons.T)
            self.network[i-1].errors = np.dot(self.network[i].synapses.T, 
                                              self.network[i].errors)

    def train(self, trainLabels, trainData, testLabels, testData, 
                    epochs, learningRate, withOutput=False):
        for i in range(epochs):
            print('\t-- Epoch {}'.format(i+1))
            for label, data in zip(trainLabels, trainData):
                target = self.oneHotEncode(label-1)
                self.feedForward(data)
                self.backpropagation(target, learningRate)
            if(withOutput):
                accuracy = self.test(testLabels, testData)
                print('Accuracy = {0:.2f}%'.format(accuracy*100))

    def test(self, labels, testData):
        correct = 0
        for i, (label, data) in enumerate(zip(labels, testData)):
            self.feedForward(data)
            bestIndex = np.argmax(self.network[-1].neurons)
            if (label == bestIndex+1):
                correct += 1
        return correct / len(labels)

    def oneHotEncode(self, index):
        vect = np.zeros((self.network[-1].count, 1))
        vect[index] = 1
        return vect

    def sigmoid(self, A):
        return 1 / (1 + np.exp(-A))

    def tanh(self, A):
        return np.tanh(A)

    def ReLU(self, A):
        return np.where(A > 0, A, 0)

    def softmax(self, A):
        e = np.exp(A - np.max(A))
        return e / e.sum()
        
    def crossEntropy(self, y, X, epsilon=1e-12):
        X = np.clip(X, epsilon, 1. - epsilon)
        N = X.shape[0]
        ce = -np.sum(y*np.log(X+1e-9))/N
        return ce
        

In [107]:
class DataLoader:
    def __init__(self):
        pass

    def standardize(self, A):
        return (A - np.mean(A)) / np.std(A)

    def normalize(self, A):
        return (A - np.min(A)) / (np.max(A) - np.min(A))

    def extractMNIST(self, fileName):
        labels = []
        fname = open(fileName, "r")
        values = fname.readlines()[:20000]
        fname.close()
        for i, record in enumerate(values):
            data = record.split(",")
            values[i] = self.standardize(np.asfarray(data[1:]))
            labels.append(int(data[0]))
        return labels, values

In [109]:
# Number of training sessions
network = [784, 100, 100, 10]
epochs = 5
learningRate = 0.01
displayOutput = True
dl = DataLoader()

# Create neural network
print("Creating Network")
ann = NeuralNetwork()
ann.initNetwork(network)

# Open file to loop through
print("Opening Training Data")
MNIST_Train_Labels, MNIST_Train_Values = dl.extractMNIST("MNIST/mnist_train.csv")
print("Opening Testing Data")
MNIST_Test_Labels, MNIST_Test_Values = dl.extractMNIST("MNIST/mnist_test.csv")

# Train
print("Training:")
ann.train(MNIST_Train_Labels, MNIST_Train_Values, MNIST_Test_Labels, MNIST_Test_Values, 
          epochs, learningRate, displayOutput)

# Test
if (not displayOutput):
    print("Testing:")
    accuracy = ann.test(MNIST_Test_Labels, MNIST_Test_Values)

    # Print Accuracy
    print("Accuracy = %.2f%%" % (accuracy * 100))

Creating Network
Opening Training Data
Opening Testing Data
Training:
	-- Epoch 1
Accuracy = 68.96%
	-- Epoch 2
Accuracy = 73.15%
	-- Epoch 3
Accuracy = 74.51%
	-- Epoch 4
Accuracy = 73.53%
	-- Epoch 5


KeyboardInterrupt: 